**Setup the Environment**

In [ ]:
# Used A100 for training and inference which takes about 5 minutes.

!git clone https://github.com/huggingface/diffusers
%cd diffusers
!pip install -e .
%cd examples/dreambooth
!pip install -r requirements.txt
!accelerate config default

**Download Dataset**

In [ ]:
from huggingface_hub import snapshot_download

local_dir = "./dog"
snapshot_download(
    "diffusers/dog-example",
    local_dir=local_dir, repo_type="dataset",
    ignore_patterns=".gitattributes",
)

**Training**

In [ ]:
from huggingface_hub import notebook_login
notebook_login()
# Go to this url and generate token, then copy it below: https://huggingface.co/settings/tokens

In [ ]:
!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path="CompVis/stable-diffusion-v1-4"  \
  --instance_data_dir="dog" \
  --class_data_dir="dog_class" \
  --output_dir="dog_model" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="a photo of sks dog" \
  --class_prompt="a photo of dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=500 \
  --max_train_steps=1000


**Inference**

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "dog_model"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

In [ ]:
prompt = "A photo of sks dog in a bucket"
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]
image.save("dog_1.png")

In [ ]:
prompt = "A photo of sks dog on the grass"
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]
image.save("dog_2.png")

In [ ]:
prompt = "A photo of sks dog in a car"
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]
image.save("dog_3.png")